# Molecular interaction preferences and the Interaction API

IsoStar is a knowledge-base of intermolecular interaction preferences derived from the CSD and PDB. This data can also be accessed _via_ the [Interaction API](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/interaction.html).

In [ ]:
%run ../Discovery_Notebook_Utils.py

In [ ]:
from ccdc.interaction import InteractionLibrary

### Configuration

### Initialization

In [ ]:
logger.info(script_info)

Initialise the IsoStar central and contact group libraries...

In [ ]:
central_lib = InteractionLibrary.CentralGroupLibrary()
contact_lib = InteractionLibrary.ContactGroupLibrary()

### Inspect the interaction group libraries

Available central groups...

In [ ]:
len(central_lib.groups)

In [ ]:
print('\n'.join([x.name for x in central_lib.groups[:10]]))

Available contact groups...

In [ ]:
len(contact_lib.groups)

In [ ]:
print('\n'.join([x.name for x in contact_lib.groups[:10]]))

We can visualise the substructure query used to define any group...

In [ ]:
group = central_lib.group_by_name('aromatic-aromatic ester')

SVG(diagram_generator.image(group.substructure_query))

In [ ]:
group = contact_lib.group_by_name('sulfoxide/sulfone O')

SVG(diagram_generator.image(group.substructure_query))

### Perform an Interaction Analysis

We will perform an interaction analyis for aliphatic ketones....

In [ ]:
central_group = central_lib.group_by_name('aliphatic-aliphatic ketone')

SVG(diagram_generator.image(central_group.substructure_query))

The contact groups for which data is available for this central group...

In [ ]:
contact_groups = [x for x in central_group.contact_groups() if x]

len(contact_groups)

We can show the data for these groups as a dataframe...

In [ ]:
def make_row(contact_group):
    
    data = central_group.interaction_data(contact_group)
        
    return [contact_group.name, data.ncontacts, *data.relative_density]

In [ ]:
def depiction(name): # Depiction of a contact-group substruture query
    
    return diagram_generator.image(contact_lib.group_by_name(name).substructure_query)

In [ ]:
contacts_df = (
    pd.DataFrame(
        data=[make_row(x) for x in contact_groups],
        columns=['Contact Group', 'No. of Contacts', 'Relative Density', 'Std. Dev.']
    )
    .sort_values('Relative Density', ascending=False)
    .assign(depiction = lambda df: df['Contact Group'].apply(depiction))
)

contacts_df.shape

In [ ]:
show_dataframe(contacts_df)

### Inspect which central and contact groups are present in a molecule

Retrieve an example molecule from the CSD (or load one from file)...

In [ ]:
refcode = 'AABHTZ'

with MoleculeReader('CSD') as reader:

    molecule = reader.molecule(refcode)

In [ ]:
SVG(diagram_generator.image(molecule))

Local utility to display a molecule with a substructure highlighted...

In [ ]:
def depict_group(group):
    
    return diagram_generator.image(molecule, highlight_atoms=group.match_atoms())

#### Central Groups

Identify the central groups in the molecule of interest...

In [ ]:
central_group_hits = central_lib.search_molecule(molecule)

len(central_group_hits)

In [ ]:
central_groups_df = pd.DataFrame([(group.name, depict_group(group)) for group in central_group_hits], columns=['Group', 'Depiction'])

show_dataframe(central_groups_df.head(3))

#### Contact Groups

Identify the contact groups in the molecule of interest...

In [ ]:
contact_group_hits = contact_lib.search_molecule(molecule)

len(contact_group_hits)

In [ ]:
contact_groups_df = pd.DataFrame([(group.name, depict_group(group)) for group in central_group_hits], columns=['Group', 'Depiction'])

show_dataframe(contact_groups_df.head(3))